In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import livenet
importlib.reload(livenet)
core = livenet.core
import simple_log
simple_log.level = simple_log.LogLevel.INFO
from simple_log import LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
var = 42
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: <object object at 0x7f2aa952b940>
2.2.1+cu121


'%.4f'

In [134]:
downscale = (14, 14)
train_x, train_y = livenet.datasets.to_plain(*livenet.datasets.get_mnist_train(), downscale=downscale, to_odd=True, to_gray=True)


In [153]:
context = core.livenet.Context()
network = livenet.nets.create_perceptron(train_x.shape[1], 2, 2, context)
batch_iterator = livenet.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = livenet.nets.criterion_classification_n
optimizer = livenet.nets.create_optimizer(network)
trainer = livenet.net_trainer.NetTrainer(network, batch_iterator, criterion, optimizer, epoch_size=10)

LiveNet


In [154]:
network.context.regularization_l1 = 0.01
optimizer.learning_rate = 0.001
trainer.step(5000)


Iˈ0.000 1.040 = 1.000+0.040reg params=16 livenet/net_trainer.py:93
Iˈ0.013 1.037 = 0.999+0.038reg params=16 livenet/net_trainer.py:93
Iˈ0.024 1.036 = 1.000+0.036reg params=16 livenet/net_trainer.py:93
Iˈ0.037 1.034 = 1.000+0.034reg params=16 livenet/net_trainer.py:93
Iˈ0.049 1.033 = 1.000+0.033reg params=16 livenet/net_trainer.py:93
Iˈ0.061 1.031 = 1.000+0.031reg params=16 livenet/net_trainer.py:93
Iˈ0.073 1.029 = 1.000+0.029reg params=16 livenet/net_trainer.py:93
Iˈ0.086 1.028 = 1.000+0.028reg params=16 livenet/net_trainer.py:93
Iˈ0.097 1.026 = 1.000+0.026reg params=16 livenet/net_trainer.py:93
Iˈ0.109 1.025 = 1.000+0.025reg params=16 livenet/net_trainer.py:93
Iˈ0.121 1.024 = 1.000+0.024reg params=16 livenet/net_trainer.py:93
Iˈ0.132 1.022 = 1.000+0.023reg params=16 livenet/net_trainer.py:93
Iˈ0.144 1.022 = 1.000+0.021reg params=16 livenet/net_trainer.py:93
Iˈ0.156 1.020 = 1.000+0.020reg params=16 livenet/net_trainer.py:93
Iˈ0.167 1.019 = 1.000+0.019reg params=16 livenet/net_trainer.p

In [155]:
pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y - pred_bin
f"accuracy {len(diff[diff == 0]) / len(diff): .3f}"


'accuracy  0.757'

In [156]:
for name, p in sorted(network.named_parameters(),key=lambda x: len(x[0]) + ord(x[0][0]) / 1000. + ord(x[0][1]) / 1e6):
    print(name, f"{p.item():.1f}")



D0 -3.7
D1 3.7
N1 1.5
N1->D0 4.0
N1->D1 -4.8
S1->N1 -3.4
S2->N1 3.1
S3->N1 1.1


In [ ]:
%matplotlib
plt.ion()

def param_amount(param):
    val = len(param["params"])
    return val

def param_max(param):
    val = np.max(np.abs(param["params"]))
    return val

def param_picker1(param):
    try:
        val0 = param["params"]["S262->D1"].item()
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_amount)
plt.grid()
#plt.plot(values)

accum = core.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
accum.plot()


D0 -14.3
D1 14.3
N1 11.9
N1->D0 -0.1
N1->D1 -3.0
S1->N1 -1.2
S2->N1 2.9
S3->N1 1.3


In [52]:
network._parameters


OrderedDict([('D0',
              Parameter containing:
              tensor(24.0669, requires_grad=True)),
             ('D1',
              Parameter containing:
              tensor(-24.0669, requires_grad=True)),
             ('N1',
              Parameter containing:
              tensor(13.0908, requires_grad=True)),
             ('S6->N1',
              Parameter containing:
              tensor(0.7347, requires_grad=True)),
             ('S8->N1',
              Parameter containing:
              tensor(-0.6229, requires_grad=True)),
             ('S9->N1',
              Parameter containing:
              tensor(-1.5580, requires_grad=True)),
             ('S11->N1',
              Parameter containing:
              tensor(-0.5902, requires_grad=True)),
             ('N1->D0',
              Parameter containing:
              tensor(-1.7404, requires_grad=True)),
             ('N1->D1',
              Parameter containing:
              tensor(1.5365, requires_grad=True))])

In [ ]:
core.livenet.export_onnx(network, "/home/spometun/model.onnx")
